<a href="https://colab.research.google.com/github/rajuzumaki2207/Time_series_Sequence/blob/main/RUL_TurboFan/4_TurboFan_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns



from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import os
plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (20,10)
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

## Import Data

In [52]:
## Train data and Test data
# define column names for easy indexing
index_names = ['id', 'cycles']
setting_names = ['op1', 'op2', 'op3']
sensor_names = ['s_{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names
train = pd.read_csv("/content/drive/MyDrive/ML/Projects/RUL_TurboFan/train_FD001.txt", sep = "\s+", names=col_names)
test = pd.read_csv("/content/drive/MyDrive/ML/Projects/RUL_TurboFan/test_FD001.txt", sep = "\s+", names =col_names)
y_test = pd.read_csv('/content/drive/MyDrive/ML/Projects/RUL_TurboFan/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

In [53]:
test_df= test.groupby('id').agg({'cycles':'max'})

In [54]:
test_df

,cycles
id,
1,31
2,49
3,126
4,106
5,98
...,...
96,97
97,134
98,121


In [55]:
y_test['cycles']=test_df['cycles'].values

In [56]:
y_test

,RUL,cycles
0,112,31
1,98,49
2,69,126
3,82,106
4,91,98
...,...,...
95,137,97
96,82,134
97,59,121
98,117,97


In [57]:
y_test["max"] = y_test["RUL"] + y_test["cycles"] 

In [58]:
y_test

,RUL,cycles,max
0,112,31,143
1,98,49,147
2,69,126,195
3,82,106,188
4,91,98,189
...,...,...,...
95,137,97,234
96,82,134,216
97,59,121,180
98,117,97,214


In [59]:
test_df['cycles']= y_test["RUL"].values + y_test["cycles"].values

In [60]:
test_df

,cycles
id,
1,143
2,147
3,195
4,188
5,189
...,...
96,234
97,216
98,180


In [61]:
train_df = train.groupby('id').agg({'cycles':'max'})

## Add RUL with add_rul function

In [62]:
def add_rul(df, max_df):
   # Get the total number of cycles for each unit
    max_cycle= max_df['cycles']
    
    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='id', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["cycles"]
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

In [63]:
## Add RUL
train = add_rul(train, train_df)

## Deep Learning for Predictive Maintaince

Among the deep learning methods, Long Short Term Memory (LSTM) networks are especially appealing to the predictive maintenance domain due to the fact that they are very good at learning from sequences. This fact lends itself to their applications using time series data by making it possible to look back for longer periods of time to detect failure patterns. In this notebook, we build an LSTM network for the data set and scenerio described at [Predictive Maintenance Template](https://gallery.azure.ai/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. In summary, the template uses simulated aircraft sensor values to predict when an aircraft engine will fail in the future so that maintenance can be planned in advance. 


This notebook uses keras deep learning library with Microsoft Cognitive Toolkit CNTK as backend.

In [64]:
w1= 30
w0 =15

train["label1"] = np.where(train["RUL"]<=w1,1,0)
train["label2"] = train["label1"]


In [65]:
train[(train["RUL"]<15) & (train["id"]==1)]

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_15,s_16,s_17,s_18,s_19,s_20,s_21,RUL,label1,label2
177,1,178,-0.0001,0.0002,100.0,518.67,643.38,1605.33,1424.65,14.62,...,8.4770,0.03,395,2388,100.0,38.65,23.1770,14,1,1
178,1,179,0.0023,-0.0003,100.0,518.67,642.86,1592.56,1429.45,14.62,...,8.4947,0.03,395,2388,100.0,38.77,23.1888,13,1,1
179,1,180,-0.0024,0.0000,100.0,518.67,643.58,1599.87,1417.14,14.62,...,8.4737,0.03,395,2388,100.0,38.86,23.0070,12,1,1
180,1,181,-0.0006,0.0005,100.0,518.67,643.44,1596.71,1420.64,14.62,...,8.4720,0.03,397,2388,100.0,38.34,23.1652,11,1,1
181,1,182,-0.0010,0.0003,100.0,518.67,644.21,1602.08,1426.62,14.62,...,8.5158,0.03,397,2388,100.0,38.43,23.1787,10,1,1
182,1,183,0.0001,0.0001,100.0,518.67,643.24,1597.23,1419.03,14.62,...,8.5041,0.03,396,2388,100.0,38.56,23.1302,9,1,1
183,1,184,0.0004,0.0003,100.0,518.67,644.07,1605.44,1432.52,14.62,...,8.5133,0.03,395,2388,100.0,38.43,23.0146,8,1,1
184,1,185,0.0008,0.0001,100.0,518.67,643.80,1603.46,1424.40,14.62,...,8.4984,0.03,395,2388,100.0,38.51,23.0700,7,1,1
185,1,186,0.0027,-0.0003,100.0,518.67,643.51,1595.16,1426.30,14.62,...,8.5227,0.03,397,2388,100.0,38.47,23.0564,6,1,1
186,1,187,-0.0047,-0.0000,100.0,518.67,643.32,1592.10,1427.27,14.62,...,8.5218,0.03,396,2388,100.0,38.42,23.0822,5,1,1


In [66]:
train.loc[train["RUL"]<=w0, "label2"]=2

In [67]:
train[(train["RUL"]<15) & (train["id"]==1)]

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_15,s_16,s_17,s_18,s_19,s_20,s_21,RUL,label1,label2
177,1,178,-0.0001,0.0002,100.0,518.67,643.38,1605.33,1424.65,14.62,...,8.4770,0.03,395,2388,100.0,38.65,23.1770,14,1,2
178,1,179,0.0023,-0.0003,100.0,518.67,642.86,1592.56,1429.45,14.62,...,8.4947,0.03,395,2388,100.0,38.77,23.1888,13,1,2
179,1,180,-0.0024,0.0000,100.0,518.67,643.58,1599.87,1417.14,14.62,...,8.4737,0.03,395,2388,100.0,38.86,23.0070,12,1,2
180,1,181,-0.0006,0.0005,100.0,518.67,643.44,1596.71,1420.64,14.62,...,8.4720,0.03,397,2388,100.0,38.34,23.1652,11,1,2
181,1,182,-0.0010,0.0003,100.0,518.67,644.21,1602.08,1426.62,14.62,...,8.5158,0.03,397,2388,100.0,38.43,23.1787,10,1,2
182,1,183,0.0001,0.0001,100.0,518.67,643.24,1597.23,1419.03,14.62,...,8.5041,0.03,396,2388,100.0,38.56,23.1302,9,1,2
183,1,184,0.0004,0.0003,100.0,518.67,644.07,1605.44,1432.52,14.62,...,8.5133,0.03,395,2388,100.0,38.43,23.0146,8,1,2
184,1,185,0.0008,0.0001,100.0,518.67,643.80,1603.46,1424.40,14.62,...,8.4984,0.03,395,2388,100.0,38.51,23.0700,7,1,2
185,1,186,0.0027,-0.0003,100.0,518.67,643.51,1595.16,1426.30,14.62,...,8.5227,0.03,397,2388,100.0,38.47,23.0564,6,1,2
186,1,187,-0.0047,-0.0000,100.0,518.67,643.32,1592.10,1427.27,14.62,...,8.5218,0.03,396,2388,100.0,38.42,23.0822,5,1,2


## Normalize the columns

In [68]:
# MinMax normalization
from sklearn.preprocessing import MinMaxScaler

train["cycle_norm"]= train["cycles"]

cols_normalize = train.columns.difference(["id", "cycles", "RUL", "label1", "label2"])


min_max_scaler = MinMaxScaler()

norm_train = pd.DataFrame(min_max_scaler.fit_transform(train[cols_normalize]),
                          columns= cols_normalize,
                          index= train.index
                          )

In [69]:
norm_train

,cycle_norm,op1,op2,op3,s_1,s_10,s_11,s_12,s_13,s_14,...,s_2,s_20,s_21,s_3,s_4,s_5,s_6,s_7,s_8,s_9
0,0.000000,0.459770,0.166667,0.0,0.0,0.0,0.369048,0.633262,0.205882,0.199608,...,0.183735,0.713178,0.724662,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755
1,0.002770,0.609195,0.250000,0.0,0.0,0.0,0.380952,0.765458,0.279412,0.162813,...,0.283133,0.666667,0.731014,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242
2,0.005540,0.252874,0.750000,0.0,0.0,0.0,0.250000,0.795309,0.220588,0.171793,...,0.343373,0.627907,0.621375,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043
3,0.008310,0.540230,0.500000,0.0,0.0,0.0,0.166667,0.889126,0.294118,0.174889,...,0.343373,0.573643,0.662386,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518
4,0.011080,0.390805,0.333333,0.0,0.0,0.0,0.255952,0.746269,0.235294,0.174734,...,0.349398,0.589147,0.704502,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,0.540166,0.477011,0.250000,0.0,0.0,0.0,0.726190,0.170576,0.558824,0.194344,...,0.686747,0.271318,0.109500,0.587312,0.782917,0.0,1.0,0.254428,0.439394,0.196491
20627,0.542936,0.408046,0.083333,0.0,0.0,0.0,0.708333,0.211087,0.500000,0.188668,...,0.701807,0.124031,0.366197,0.729453,0.866475,0.0,1.0,0.162641,0.500000,0.194651
20628,0.545706,0.522989,0.500000,0.0,0.0,0.0,0.738095,0.281450,0.529412,0.212148,...,0.665663,0.232558,0.053991,0.684979,0.775321,0.0,1.0,0.175523,0.515152,0.198196
20629,0.548476,0.436782,0.750000,0.0,0.0,0.0,0.916667,0.208955,0.514706,0.203065,...,0.608434,0.116279,0.234466,0.746021,0.747468,0.0,1.0,0.133655,0.530303,0.233285


In [70]:
join_df = train[train.columns.difference(cols_normalize)].join(norm_train)

In [71]:
train_df = join_df.reindex(columns= train.columns)
train_df.head()

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_16,s_17,s_18,s_19,s_20,s_21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [72]:
## test
test["cycle_norm"]= test["cycles"]

norm_test = pd.DataFrame(min_max_scaler.fit_transform(test[cols_normalize]),
                          columns= cols_normalize,
                          index= test.index
                          )
join_df = test[test.columns.difference(cols_normalize)].join(norm_test)
test_df = join_df.reindex(columns= test.columns)
test_df.head()

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm
0,1,1,0.65625,0.692308,0.0,0.0,0.596215,0.421968,0.282214,0.0,...,0.325581,0.152259,0.347076,0.0,0.375,0.0,0.0,0.500000,0.620099,0.000000
1,1,2,0.34375,0.230769,0.0,0.0,0.182965,0.504025,0.225240,0.0,...,0.395349,0.277907,0.227709,0.0,0.500,0.0,0.0,0.645455,0.645718,0.003311
2,1,3,0.53125,0.538462,0.0,0.0,0.419558,0.464814,0.346130,0.0,...,0.325581,0.192892,0.533557,0.0,0.500,0.0,0.0,0.700000,0.681104,0.006623
3,1,4,0.77500,0.461538,0.0,0.0,0.413249,0.391587,0.449867,0.0,...,0.372093,0.217896,0.282359,0.0,0.250,0.0,0.0,0.627273,0.620382,0.009934
4,1,5,0.60000,0.461538,0.0,0.0,0.435331,0.471306,0.357974,0.0,...,0.325581,0.187891,0.337009,0.0,0.125,0.0,0.0,0.618182,0.676008,0.013245


In [73]:
test_df

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm
0,1,1,0.65625,0.692308,0.0,0.0,0.596215,0.421968,0.282214,0.0,...,0.325581,0.152259,0.347076,0.0,0.375,0.0,0.0,0.500000,0.620099,0.000000
1,1,2,0.34375,0.230769,0.0,0.0,0.182965,0.504025,0.225240,0.0,...,0.395349,0.277907,0.227709,0.0,0.500,0.0,0.0,0.645455,0.645718,0.003311
2,1,3,0.53125,0.538462,0.0,0.0,0.419558,0.464814,0.346130,0.0,...,0.325581,0.192892,0.533557,0.0,0.500,0.0,0.0,0.700000,0.681104,0.006623
3,1,4,0.77500,0.461538,0.0,0.0,0.413249,0.391587,0.449867,0.0,...,0.372093,0.217896,0.282359,0.0,0.250,0.0,0.0,0.627273,0.620382,0.009934
4,1,5,0.60000,0.461538,0.0,0.0,0.435331,0.471306,0.357974,0.0,...,0.325581,0.187891,0.337009,0.0,0.125,0.0,0.0,0.618182,0.676008,0.013245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.81875,0.461538,0.0,0.0,0.665615,0.789665,0.641209,0.0,...,0.255814,0.935703,0.664909,0.0,0.625,0.0,0.0,0.309091,0.370842,0.639073
13092,100,195,0.44375,0.384615,0.0,0.0,0.659306,0.692028,0.769042,0.0,...,0.465116,0.914003,0.567593,0.0,0.750,0.0,0.0,0.236364,0.483652,0.642384
13093,100,196,0.47500,0.230769,0.0,0.0,0.728707,0.626071,0.458036,0.0,...,0.348837,0.971066,0.594919,0.0,0.750,0.0,0.0,0.281818,0.381741,0.645695
13094,100,197,0.27500,0.538462,0.0,0.0,0.671924,0.673851,0.714111,0.0,...,0.441860,1.000000,0.662991,0.0,0.750,0.0,0.0,0.318182,0.473461,0.649007


In [74]:
y_test

,RUL,cycles,max
0,112,31,143
1,98,49,147
2,69,126,195
3,82,106,188
4,91,98,189
...,...,...,...
95,137,97,234
96,82,134,216
97,59,121,180
98,117,97,214


In [75]:
y_test["id"]= y_test.index+1
g_truth = y_test.drop(["cycles", "RUL"], axis=1)


In [76]:
g_truth.columns = ["max", "id"]
g_truth

,max,id
0,143,1
1,147,2
2,195,3
3,188,4
4,189,5
...,...,...
95,234,96
96,216,97
97,180,98
98,214,99


In [77]:
test_df = test_df.merge(g_truth, on=["id"], how="left")
test_df.head()

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm,max
0,1,1,0.65625,0.692308,0.0,0.0,0.596215,0.421968,0.282214,0.0,...,0.152259,0.347076,0.0,0.375,0.0,0.0,0.500000,0.620099,0.000000,143
1,1,2,0.34375,0.230769,0.0,0.0,0.182965,0.504025,0.225240,0.0,...,0.277907,0.227709,0.0,0.500,0.0,0.0,0.645455,0.645718,0.003311,143
2,1,3,0.53125,0.538462,0.0,0.0,0.419558,0.464814,0.346130,0.0,...,0.192892,0.533557,0.0,0.500,0.0,0.0,0.700000,0.681104,0.006623,143
3,1,4,0.77500,0.461538,0.0,0.0,0.413249,0.391587,0.449867,0.0,...,0.217896,0.282359,0.0,0.250,0.0,0.0,0.627273,0.620382,0.009934,143
4,1,5,0.60000,0.461538,0.0,0.0,0.435331,0.471306,0.357974,0.0,...,0.187891,0.337009,0.0,0.125,0.0,0.0,0.618182,0.676008,0.013245,143


In [78]:
test_df["RUL"] = test_df["max"]- test_df["cycles"]

In [79]:
test_df.drop("max", axis =1, inplace=True)
test_df.head()

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm,RUL
0,1,1,0.65625,0.692308,0.0,0.0,0.596215,0.421968,0.282214,0.0,...,0.152259,0.347076,0.0,0.375,0.0,0.0,0.500000,0.620099,0.000000,142
1,1,2,0.34375,0.230769,0.0,0.0,0.182965,0.504025,0.225240,0.0,...,0.277907,0.227709,0.0,0.500,0.0,0.0,0.645455,0.645718,0.003311,141
2,1,3,0.53125,0.538462,0.0,0.0,0.419558,0.464814,0.346130,0.0,...,0.192892,0.533557,0.0,0.500,0.0,0.0,0.700000,0.681104,0.006623,140
3,1,4,0.77500,0.461538,0.0,0.0,0.413249,0.391587,0.449867,0.0,...,0.217896,0.282359,0.0,0.250,0.0,0.0,0.627273,0.620382,0.009934,139
4,1,5,0.60000,0.461538,0.0,0.0,0.435331,0.471306,0.357974,0.0,...,0.187891,0.337009,0.0,0.125,0.0,0.0,0.618182,0.676008,0.013245,138


In [80]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

,id,cycles,op1,op2,op3,s_1,s_2,s_3,s_4,s_5,...,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm,RUL,label1,label2
0,1,1,0.65625,0.692308,0.0,0.0,0.596215,0.421968,0.282214,0.0,...,0.0,0.375,0.0,0.0,0.500000,0.620099,0.000000,142,0,0
1,1,2,0.34375,0.230769,0.0,0.0,0.182965,0.504025,0.225240,0.0,...,0.0,0.500,0.0,0.0,0.645455,0.645718,0.003311,141,0,0
2,1,3,0.53125,0.538462,0.0,0.0,0.419558,0.464814,0.346130,0.0,...,0.0,0.500,0.0,0.0,0.700000,0.681104,0.006623,140,0,0
3,1,4,0.77500,0.461538,0.0,0.0,0.413249,0.391587,0.449867,0.0,...,0.0,0.250,0.0,0.0,0.627273,0.620382,0.009934,139,0,0
4,1,5,0.60000,0.461538,0.0,0.0,0.435331,0.471306,0.357974,0.0,...,0.0,0.125,0.0,0.0,0.618182,0.676008,0.013245,138,0,0


## Modelling

Traditional predictive maintaince machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.


For using LSTMs in time series domain,one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the Predictive Maintenance Template which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

In [81]:
## pick a large window size of 50 cycles

sequence_length = 50


In [82]:
## preparing data for visualizations
##
engine_id3 = test_df[test_df['id'] == 3]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]

In [88]:
def gen_sequence(id, seq_length, seq_cols):
  """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """

  data_array = id[seq_cols].values
  num_elements = data_array.shape[0]

  for start, stop in zip(range(0, num_elements-seq_length), range(seq_length,num_elements)):
    yield data_array[start:stop,:]

In [119]:
index_names = ['id', 'cycles']
setting_names = ['op1', 'op2', 'op3',"cycle_norm"]
sensor_names = ['s_{}'.format(i) for i in range(1,22)] 
seq_col= setting_names + sensor_names
xx=(train_df[train_df['id']==3],50, seq_col) ## test for engine ID 3

In [120]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, seq_col)) 
           for id in train_df['id'].unique())

In [121]:
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

In [122]:
seq_array.shape

(15631, 50, 25)

In [123]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [124]:
## generate labels

label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

## Lets build an LSTM Network

In [131]:
num_of_features = seq_array.shape[2]

num_of_out = label_array.shape[1]



In [139]:
from tensorflow import keras

from tensorflow.keras.layers import Dropout, LSTM, Dense

model = keras.Sequential()
  
model.add(LSTM(
    input_shape = (sequence_length, num_of_features),
    units = 128,
    return_sequences =True
    ))
model.add(Dropout(0.2))
model.add(LSTM(
    units=64,
    return_sequences =False

))

model.add(Dropout(0.2))

model.add(Dense(units =num_of_out,activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [140]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50, 128)           78848     
                                                                 
 dropout_2 (Dropout)         (None, 50, 128)           0         
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 128,321
Trainable params: 128,321
Non-trainable params: 0
_________________________________________________________________
None


In [147]:
from keras import callbacks


%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
Epoch 1/10
75/75 [==============================] - 4s 47ms/step - loss: 0.0829 - accuracy: 0.9654 - val_loss: 0.1069 - val_accuracy: 0.9463
Epoch 2/10
75/75 [==============================] - 2s 32ms/step - loss: 0.0783 - accuracy: 0.9673 - val_loss: 0.0400 - val_accuracy: 0.9834
Epoch 3/10
75/75 [==============================] - 2s 29ms/step - loss: 0.0677 - accuracy: 0.9712 - val_loss: 0.0356 - val_accuracy: 0.9834
Epoch 4/10
75/75 [==============================] - 2s 29ms/step - loss: 0.0593 - accuracy: 0.9746 - val_loss: 0.0571 - val_accuracy: 0.9757
Epoch 5/10
75/75 [==============================] - 2s 29ms/step - loss: 0.0734 - accuracy: 0.9708 - val_loss: 0.0412 - val_accuracy: 0.9872
Epoch 6/10
75/75 [==============================] - 2s 29ms/step - loss: 0.0674 - accuracy: 0.9705 - val_loss: 0.0670 - val_accuracy: 0.9706


In [149]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

79/79 [==============================] - 1s 14ms/step - loss: 0.0693 - accuracy: 0.9708
Accurracy: 0.9708272218704224


In [155]:
# make predictions and compute confusion matrix
from sklearn.metrics import confusion_matrix, recall_score, precision_score

y_pred = (model.predict(seq_array,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

79/79 [==============================] - 1s 10ms/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12394,   137],
       [  319,  2781]])

In [159]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.953050034270048 
 recall =  0.8970967741935484
